In [ ]:
from flask import Flask, render_template, request, redirect, url_for
from flask_sqlalchemy import SQLAlchemy
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from keras.models import Sequential, load_model
from keras.layers import Dense
from google.cloud import storage
import uuid

app = Flask(__name__)

# Replace the database URI with the appropriate URI for your global database management system
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///your_database.db'
# Ensure the following configurations are set according to your database system
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SECRET_KEY'] = 'your_secret_key_here'  # Secret key for Flask sessions

db = SQLAlchemy(app)

# User model
class User(db.Model):
    id = db.Column(db.String(36), primary_key=True, unique=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    email = db.Column(db.String(120), unique=True, nullable=False)
    password = db.Column(db.String(80), nullable=False)

    def __repr__(self):
        return '<User %r>' % self.username

# Route for user registration
@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        username = request.form['username']
        email = request.form['email']
        password = request.form['password']

        # Check if the email is already registered
        if User.query.filter_by(email=email).first():
            return render_template('register.html', message='Email already registered. Please use a different email.')

        # Generate a unique user ID using UUID
        user_id = str(uuid.uuid4())

        # Create a new user record
        new_user = User(id=user_id, username=username, email=email, password=password)
        db.session.add(new_user)
        db.session.commit()

        return redirect(url_for('login'))  # Redirect to login page after registration
    return render_template('register.html')

# Route for user login
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        email = request.form['email']
        password = request.form['password']

        # Check if the user exists and the password is correct
        user = User.query.filter_by(email=email, password=password).first()
        if user:
            # Fetch user-specific data for the logged-in user
            user_data = get_user_data_by_id(user.id)

            # Load the pretrained model
            pretrained_model = load_model('global_model.keras')

            # Fine-tune the model based on user-specific data
            fine_tuned_model = fine_tune_model(pretrained_model, user_data)

            # Save the fine-tuned model locally
            filename = save_model_locally(fine_tuned_model, user.id)

            # Upload the fine-tuned model to Google Cloud Storage
            upload_model_to_cloud(filename, 'your_bucket_name', f'fine_tuned_model_{user.id}.keras')

            return redirect(url_for('dashboard'))  # Redirect to dashboard after login
        else:
            return render_template('login.html', message='Invalid email or password. Please try again.')
    return render_template('login.html')

def get_user_data_by_id(user_id):
    # Query the database for user-specific data
    motion_data = MotionData.query.filter_by(user_id=user_id).all()
    image_data = ImageData.query.filter_by(user_id=user_id).all()
    tap_data = Tap.query.filter_by(user_id=user_id).all()
    phonetic_complexity_data = phonetic_complexity.query.filter_by(user_id=user_id).all()
    phonetic_probability_data = phonetic_probability.query.filter_by(user_id=user_id).all()
    iconizity_data = iconizity.query.filter_by(user_id=user_id).all()

    # Process and convert data to NumPy arrays
    motion_data = np.array([[entry.accelerometer_x, entry.accelerometer_y, entry.accelerometer_z, entry.attribute1, entry.attribute2, ...] for entry in motion_data])
    image_data = np.array([[entry.handshape, entry.thumb_position, entry.minor_location, entry.major_location, entry.attribute1, entry.attribute2, ...] for entry in image_data])
    tap_data = np.array([[entry.word, entry.sentence, entry.attribute1, entry.attribute2, ...] for entry in tap_data])
    phonetic_complexity_data = np.array([[entry.attribute1, entry.attribute2, ...] for entry in phonetic_complexity_data])
    phonetic_probability_data = np.array([[entry.attribute1, entry.attribute2, ...] for entry in phonetic_probability_data])
    iconizity_data = np.array([[entry.attribute1, entry.attribute2, ...] for entry in iconizity_data])

    # Perform one-hot encoding for tap data if available
    if tap_data:
        encoder = OneHotEncoder()
        tap_encoded = encoder.fit_transform(tap_data).toarray()
    else:
        tap_encoded = None

    # Check if any of the required data is missing
    if not motion_data.size:
        motion_data = None
    if not image_data.size:
        image_data = None
    if not phonetic_complexity_data.size:
        phonetic_complexity_data = None
    if not phonetic_probability_data.size:
        phonetic_probability_data = None
    if not iconizity_data.size:
        iconizity_data = None

    return motion_data, image_data, tap_encoded, phonetic_complexity_data, phonetic_probability_data, iconizity_data

# Fine-tune the model based on user-specific data
def fine_tune_model(model, user_data):
    motion, imagesign, tap, phonetic_complexity_data, phonetic_probability_data, iconizity_data = user_data

    # Combine motion, imagesign, and tap data
    X = np.concatenate((motion, imagesign, tap, phonetic_complexity_data, phonetic_probability_data, iconizity_data), axis=1)

    # Standardize the numerical features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Predict cluster labels using the pretrained model
    cluster_labels = model.predict_classes(X_scaled)

    # Define your Keras model for fine-tuning
    # Example architecture, adjust as needed
    fine_tuned_model = Sequential([
        Dense(64, activation='relu', input_shape=(X_scaled.shape[1],)),
        Dense(32, activation='relu'),
        Dense(np.max(cluster_labels) + 1, activation='softmax')  # Use softmax activation for multi-class classification
    ])

    # Compile your fine-tuned model
    fine_tuned_model.compile(optimizer='adam',
                              loss='sparse_categorical_crossentropy',  # Use sparse categorical cross-entropy for integer labels
                              metrics=['accuracy'])  # Use accuracy as a metric

    # Fine-tune your model
    fine_tuned_model.fit(X_scaled, cluster_labels, epochs=10, batch_size=32, validation_split=0.2)

    return fine_tuned_model

# Function to save the fine-tuned model locally
def save_model_locally(model, user_id):
    filename = f'fine_tuned_model_cluster_{user_id}.keras'
    model.save(filename)
    return filename

# Function to upload the fine-tuned model to Google Cloud Storage
def upload_model_to_cloud(filename, bucket_name, destination_blob_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(filename)
    print(f"Model uploaded to cloud storage: gs://{bucket_name}/{destination_blob_name}")

if __name__ == '__main__':
    app.run(debug=True)

In order to make that work, the "def fetch_user_specific_data_for_current_user()" has to be integrated in authentication mechanism. This example scripr covers registration, the triggered unique identifier, and also login case, plus fine-tining. Phonetic complexity, ph. probability, iconizity, tap, motion, and image sign data are checked for availability and set to "None" if not available, hence skipped in that case. 